In [5]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  # Ensure this import is included
from bs4 import BeautifulSoup
import time
import pandas as pd

def scrape_job_listings(query):
    chrome_driver_path = "/Users/roshini/Downloads/chromedriver-mac-arm64/chromedriver"
    service = Service(chrome_driver_path)
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument(f"user-agent={user_agent}")
    # Initialize the Chrome driver with the service object
    driver = webdriver.Chrome(options=chrome_options,service=service)
    search_query = query
    start = 0
    limit = 15
    max_pages = 30
    job_titles = []
    company_names = []
    job_details_list = []
    place_list = []
    experience_list = []
    skills_list = []
    while start < max_pages * limit:
        url = f"https://www.foundit.in/srp/results?start={start}&sort=1&limit={limit}&query={search_query}"
        driver.get(url)
        time.sleep(5)
        html = driver.page_source
        # Parsing the HTML
        soup = BeautifulSoup(html, "html.parser")
        results = soup.find_all("div", class_="srpResultCardContainer")
        for result in results:
            job_title = result.find('div', class_='jobTitle').get_text().strip()
            company_name = result.find('div', class_='companyName').get_text().strip()
            # Extract other details similarly using the appropriate classes and HTML structure
            job_details = result.find('div', class_='cardBody').find("div", class_="bodyRow").find("div", class_="details").get_text().strip()
            place = result.find("div", class_="cardBody").find_all("div", class_="bodyRow")[1].find("div", class_="details").get_text().strip()
            experience = result.find("div", class_="cardBody").find_all("div", class_="bodyRow")[2].find("div", class_="details").get_text().strip()
            skill_details = result.find("div",class_="skillDetails")
            skills = []
            if skill_details:
                skill_container = skill_details.find_all("div",class_="skill")
                for skill in skill_container:
                    skill_name = skill.find("div", class_="skillTitle").get_text(strip=True)
                    skills.append(skill_name)
            job_titles.append(job_title)
            company_names.append(company_name)
            job_details_list.append(job_details)
            place_list.append(place)
            experience_list.append(experience)
            skills_list.append(','.join(skills))
        start = start+limit
    driver.quit()
    data = {
        'Job Title': job_titles,
        'Company Name': company_names,
        'Job Details': job_details_list,
        'Place of Work': place_list,
        'Work Experience': experience_list,
        'Skills': skills_list
    }
    df = pd.DataFrame(data)
    return df

query = "python"
output_file = f'{query}Listings.xlsx'
df = scrape_job_listings(query)
df.to_excel(output_file, index=False)


    
